# CSV + API

In this reboot, we are going to use:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal of this livecode is to load the data from a CSV + loop over rows to enrich each row with information such as:

- List of subjects (Science, Humor, Travel, etc.)
- The cover URL of the book
- Other information you'd find useful in the JSON API

First, download the CSV in the local folder:

In [1]:
!curl -L https://gist.githubusercontent.com/ssaunier/351b17f5a7a009808b60aeacd1f4a036/raw/books.csv > books.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1509k  100 1509k    0     0  3900k      0 --:--:-- --:--:-- --:--:-- 3899k


In [2]:
!ls -lh

total 4256
-rw-r--r--@ 1 bouchrakostet  staff   579B Jul  9 22:08 README.md
-rw-r--r--@ 1 bouchrakostet  staff   9.2K Jul  9 22:20 Recap.ipynb
-rw-r--r--@ 1 bouchrakostet  staff   1.5M Jul  9 22:20 books.csv


Then import the usual suspects!

In [3]:
import requests
import pandas as pd
import numpy as np

## Load books from CSV

In [4]:
books_df = pd.read_csv('books.csv', on_bad_lines='skip')
books_df = books_df.drop(columns=['bookID', 'isbn', 'average_rating', 'language_code', 'ratings_count', 'text_reviews_count'])
books_df

,title,authors,isbn13,# num_pages
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,9780439785969,652
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,9780439358071,870
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,9780439554930,320
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893,352
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,9780439655484,435
...,...,...,...,...
13714,M Is for Magic,Neil Gaiman-Teddy Kristiansen,9780061186424,260
13715,Black Orchid,Neil Gaiman-Dave McKean,9780930289553,160
13716,InterWorld (InterWorld #1),Neil Gaiman-Michael Reaves,9780061238963,239
13717,The Faeries' Oracle,Brian Froud-Jessica Macbeth,9780743201117,224


In [5]:
books_df.dtypes

title          object
authors        object
isbn13          int64
# num_pages     int64
dtype: object

Let's add a new column

In [6]:
books_df['cover_url'] = None
books_df.head()

,title,authors,isbn13,# num_pages,cover_url
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,9780439785969,652,None
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,9780439358071,870,None
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,9780439554930,320,None
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893,352,None
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,9780439655484,435,None


## API - Open Library

In [7]:
def fetch_books(isbns):
    # Define the base URL for the OpenLibrary Books API
    url = "https://openlibrary.org/api/books"

    # Create a comma-separated string like "ISBN:12345,ISBN:67890" from the list of ISBNs
    bibkeys = ",".join([f"ISBN:{isbn}" for isbn in isbns])
    
    # Define the query parameters for the API request:
    # - bibkeys: the books we want info on
    # - format: return data in JSON
    # - jscmd: 'data' returns a detailed version (titles, authors, etc.)
    params = {
        'bibkeys': bibkeys,
        'format': 'json',
        'jscmd': 'data'
    }
    
    # Send a GET request to the API with the parameters and parse the JSON response
    response = requests.get(url, params=params).json()
    
    # Return the resulting dictionary containing book data
    return response


In [8]:
%%time  # Measure how long this block of code takes to run

# Loop through the first 15 rows of the books_df DataFrame
for index, row in books_df.head(15).iterrows():
    
    # If there's no cover URL for the book, we try to fetch it
    if row['cover_url'] is None:
        isbn = row['isbn13']  # Get the ISBN-13 for the current book
        print(f"Fetching cover for {row['title']}")  # Log which book we're processing
        
        book = fetch_book(isbn)  # Call the API to fetch book details using its ISBN
        
        # If the API returned data, extract the large cover URL if available
        if book:
            cover_url = book.get('cover', {}).get('large', '')  # Use .get() to avoid KeyErrors
            books_df.loc[index, 'cover_url'] = cover_url  # Update the DataFrame with the cover URL
        else:
            books_df.loc[index, 'cover_url'] = ''  # If no data, set the cover URL as empty string


UsageError: Can't use statement directly after '%%time'!


In [9]:
books_df.head(15)

,title,authors,isbn13,# num_pages,cover_url
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,9780439785969,652,None
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,9780439358071,870,None
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,9780439554930,320,None
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893,352,None
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,9780439655484,435,None
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,9780439682589,2690,None
6,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,9780976540601,152,None
7,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,9780439827607,3342,None
8,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,9780517226957,815,None
9,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,9780345453747,815,None


## Calling the API with multiple ISBNs at a time

In [10]:
isbns = [9780439785969, 9780439358071, 9780439554930]
[f"ISBN:{isbn}" for isbn in isbns]

['ISBN:9780439785969', 'ISBN:9780439358071', 'ISBN:9780439554930']

In [11]:
",".join([f"ISBN:{isbn}" for isbn in isbns])

'ISBN:9780439785969,ISBN:9780439358071,ISBN:9780439554930'

In [12]:
def fetch_books(isbns):
    # Define the base URL of the OpenLibrary Books API
    url = "https://openlibrary.org/api/books"

    # Format the list of ISBNs into a single comma-separated string:
    # For example: ['123', '456'] → 'ISBN:123,ISBN:456'
    bibkeys = ",".join([f"ISBN:{isbn}" for isbn in isbns])
    
    # Define the parameters to send with the GET request:
    # - 'bibkeys': the ISBNs of the books we want
    # - 'format': specify the response format (JSON)
    # - 'jscmd': specify what kind of data we want (detailed info)
    params = {
        'bibkeys': bibkeys,
        'format': 'json',
        'jscmd': 'data'
    }
    
    # Send the GET request to the API with the given parameters
    # Convert the response to a JSON dictionary
    response = requests.get(url, params=params).json()
    
    # Return the dictionary containing book information
    return response


In [13]:
books_df.set_index("isbn13", inplace=True)

In [14]:
books_df.head()

,title,authors,# num_pages,cover_url
isbn13,,,,
9780439785969,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,652,None
9780439358071,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,870,None
9780439554930,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,320,None
9780439554893,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,352,None
9780439655484,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,435,None


In [15]:
!pip install tqdm

In [19]:
%%time

from tqdm import tqdm  # Import tqdm to show a progress bar during the loop

# Split the first 100 rows of books_df into 5 groups (each group has ~20 rows)
# This prevents overloading the API with too many ISBNs at once
for group in tqdm(np.array_split(books_df.head(100), 5)):

    # Fetch book data from the OpenLibrary API for the ISBNs in this group
    # The index of books_df is assumed to be the ISBN-13 codes
    books = fetch_books(list(group.index))

    # Loop through each book returned from the API
    for isbn_code, book in books.items():
        # The API returns keys like 'ISBN:0451526538', so we remove the prefix and convert to int
        isbn = int(isbn_code.strip("ISBN:"))

        # Extract the large cover URL from the API response (if it exists), or use an empty string
        cover_url = book.get("cover", {}).get("large", "")

        # Update the 'cover_url' column in books_df for this specific ISBN
        # Saved the new cover URLs directly back into your books_df
        # In pandas, .loc[row_index, column_name] = value is an in-place update
        # So unless you've made a .copy() of books_df earlier, this line updates the original DataFrame.
        books_df.loc[isbn, "cover_url"] = cover_url

/Users/bouchrakostet/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|█████████████████████████████████████████████| 5/5 [00:08<00:00,  1.65s/it]

CPU times: user 118 ms, sys: 22.2 ms, total: 140 ms
Wall time: 8.26 s


In [17]:
books_df.head(20)

,title,authors,# num_pages,cover_url
isbn13,,,,
9780439785969,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,652,None
9780439358071,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,870,None
9780439554930,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,320,None
9780439554893,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,352,None
9780439655484,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,435,None
9780439682589,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,2690,None
9780976540601,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,152,None
9780439827607,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,3342,None
9780517226957,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,815,None
